In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.__version__

'1.1.3'

## 目录

**一、长宽表的变形**

* 1.1. pivot
* 1.2. pivot_table
* 1.3. melt
* 1.4. wide_to_long


**二、索引的变形**

* 2.1. stack与unstack
* 2.2. 聚合与变形的关系


**三、其他变形函数**

* 3.1. crosstab
* 3.2. explode
* 3.3. get_dummies

**四、练习**

* Ex1：美国非法药物数据集
* Ex2：特殊的wide_to_long方法



## 正式学习内容


### 一、长宽表的变形

**长宽表的概念：**
* 长表—行数比较多
* 宽表—列数比较多

到底用long的状态存储还是wide的状态存储，取决于实际的情况，因此pandas里面也设计了相关的长宽表变形操作。

####  1.1. pivot

pivot是典型的长表变宽表的函数。

* **df.pivot**(index=None, columns=None, values=None)

参数说明：
> index: 新生成表的行索引   
> columns：新生成的表的列索引  
> values：新生成表的值



* 下表存储了张三和李四的语文和数学分数，现在想要把语文和数学分数作为列来展示**【例1】**.

* pivot的使用遵循唯一性的原则，也就是说如果我们想要 index+columns的组合必须唯一，从我们的例子理解就是 张三（姓名）+语文（学科） 这个组合在原表里只能出现一次，不然转化函数会报错**【例2】**


* pivot 三个参数被允许设定为列表，这意味着我们可以转化得到多级索引的表格。**【例3】**

In [2]:
# 原表
df = pd.DataFrame({'Class':[1,1,2,2],
                  'Name':['San Zhang','San Zhang','Si Li','Si Li'],
                  'Subject':['Chinese','Math','Chinese','Math'],
                  'Grade':[80,75,90,85]})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [4]:
# 例1
# 语文和数学分数作为列来展示
df.pivot(index = 'Name',
         columns='Subject',
         values='Grade')

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


In [6]:
# 例2

df.loc[1, 'Subject'] = 'Chinese'
try:
    df.pivot(index='Name', columns='Subject', values='Grade')
except Exception as e:
    Err_msg = e
    
Err_msg

ValueError('Index contains duplicate entries, cannot reshape')

In [26]:
# 例3

df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                            'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                  'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                               'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})

df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [27]:
# 测试类型和科目联合组成的四个类别（期中语文、期末语文、期中数学、期末数学）转到列索引
# 同时统计成绩和排名

pivot_mul = df.pivot(index=['Class','Name'],
                    columns=['Subject','Examination'],
                    values=['Grade','rank'])
pivot_mul

Grade                     rank                 
Subject         Chinese       Math       Chinese       Math      
Examination         Mid Final  Mid Final     Mid Final  Mid Final
Class Name                                                       
1     San Zhang      80    75   90    85      10    15   20     7
2     Si Li          85    65   92    88      21    15    6     2


####  1.2. pivot_table

合并后的表需要通过聚合函数操作，使相同组合的列将多组合的值变为一个值，此时就要用到 ** pivot_table**

* **pivot_table**(values=None,index=None,columns=None,aggfunc='mean',fill_value=None,margins=False,dropna=True,margins_name='All',observed=False,)


    参数说明

> values=None, 新生表的值
> index=None,  新生表的行索引 
> columns=None, 新生表的列索引
> aggfunc='mean',  相同组合合并列的方式
> margins=False,  合并所有的行和列，边际汇总
> dropna=True,  丢弃空值
> margins_name='All',  边际汇总的名称

* 组合数据不唯一，**【例1】** 张三和李四都参加了两次语文考试和数学考试，按照学院规定，最后的成绩是两次考试分数的平均值


* aggfunc支持自定义传入函数进行合并操作**【例2】**


* margins=False，表示进行边际合并即汇总所有数据 **【例3】**

In [4]:
# 

df = pd.DataFrame({'Class':[1, 1, 2, 2, 1, 1, 2, 2],
                   'Name':['San Zhang', 'San Zhang', 'Si Li', 'Si Li',
                            'San Zhang', 'San Zhang', 'Si Li', 'Si Li'],
                   'Examination': ['Mid', 'Final', 'Mid', 'Final',
                                  'Mid', 'Final', 'Mid', 'Final'],
                   'Subject':['Chinese', 'Chinese', 'Chinese', 'Chinese',
                               'Math', 'Math', 'Math', 'Math'],
                   'Grade':[80, 75, 85, 65, 90, 85, 92, 88],
                   'rank':[10, 15, 21, 15, 20, 7, 6, 2]})

df

,Class,Name,Examination,Subject,Grade,rank
0,1,San Zhang,Mid,Chinese,80,10
1,1,San Zhang,Final,Chinese,75,15
2,2,Si Li,Mid,Chinese,85,21
3,2,Si Li,Final,Chinese,65,15
4,1,San Zhang,Mid,Math,90,20
5,1,San Zhang,Final,Math,85,7
6,2,Si Li,Mid,Math,92,6
7,2,Si Li,Final,Math,88,2


In [8]:
# 例1
df.pivot_table(index = 'Name',
              columns='Subject',
              values = 'Grade',
              aggfunc = 'mean',
              )

Subject,Chinese,Math
Name,,
San Zhang,77.5,87.5
Si Li,75.0,90.0


In [7]:
# 例2
df.pivot_table(index = 'Name',
              columns='Subject',
              values = 'Grade',
              aggfunc = lambda x:x.max()
              )

Subject,Chinese,Math
Name,,
San Zhang,80,90
Si Li,85,92


In [9]:
# 例3

df.pivot_table(index = 'Name',
              columns='Subject',
              values = 'Grade',
              aggfunc = 'mean',
              margins = True
              )

Subject,Chinese,Math,All
Name,,,
San Zhang,77.50,87.50,82.5
Si Li,75.00,90.00,82.5
All,76.25,88.75,82.5


####  1.3. melt

前面介绍了pviot将长表转化为宽表，而melt可以将宽表转为长表。


* **df.melt**( id_vars=None,value_vars=None,var_name=None,value_name='value',col_level=None,)


    参数说明

> id_vars=None, 用来确定身份的列   
> value_vars=None, 需要变成长表的列，可以理解为需要合并成新的一列数据的列 ，没有指定的话，则选择所有的列  
> var_name=None,  新生成长表的列  
> value_name='value',  新生成长表的数据的值  
> col_level=None,   

* melt压缩多列到同一列**【例1】**

* melt 和 pivot 是一组互逆过程，那么就一定可以通过 pivot 操作把 df_melted 转回 df 的形式**【例2】**

In [14]:
# 
df = pd.DataFrame({'class':[1,2],
                  'name':['San Zhang', 'Si Li'],
                  'chinese':[80, 90],
                  'math':[80,75]})

df

,class,name,chinese,math
0,1,San Zhang,80,80
1,2,Si Li,90,75


In [17]:
# 例1

df_melted = df.melt(id_vars = ['class','name'],
                   value_vars=['chinese','math'],
                   var_name=['subject'],
                   value_name='grade')
df_melted


,class,name,subject,grade
0,1,San Zhang,chinese,80
1,2,Si Li,chinese,90
2,1,San Zhang,math,80
3,2,Si Li,math,75


In [23]:
# 例2

df_unmelted = df_melted.pivot(index = 'name',
                              columns='subject',
                              values='grade',
                             )
df_unmelted

subject,chinese,math
name,,
San Zhang,80,80
Si Li,90,75


####  1.4. wide_to_long

这个函数拆分的过程有点复杂，先放在这里，后面复习的时候补上

### 二、索引的变形

这一节讲的行列索引的变换。

#### 2.1. stack与unstack

* **df.unstack**(level=-1) 行索引转为列索引  
    使用说明：
    * 行索引转为列索引时，默认转化最内层 **【例1】**
    * 行索引支持同时转化多个层 **【例2】**
    * unstack 必须保证 <font color = red >被转为列索引的行索引层</font> 和 <font color = red >被保留的行索引层 </font>构成的组合是唯一的 **【例3】**

* **df.stack**(level=-1, dropna=True) 列索引转为行索引  
    使用说明：
    * 用法与unstack完全一致 **【例4】**

In [32]:
#
classes = [('A', 'cat', 'big'),
           ('A', 'dog', 'small'),
           ('B', 'cat', 'big'),
           ('B', 'dog', 'small')]

m_index1=pd.MultiIndex.from_tuples(classes)
df = pd.DataFrame(np.ones((4,2)),
                 index = m_index1,
                 columns = ['col_1','col_2'])
                
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
  dog small    1.0    1.0

In [33]:
# 例1

df.unstack()

col_1       col_2      
        big small   big small
A cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0
B cat   1.0   NaN   1.0   NaN
  dog   NaN   1.0   NaN   1.0

In [34]:
# 例2
df.unstack([0,2])

col_1                  col_2                 
        A          B           A          B      
      big small  big small   big small  big small
cat   1.0   NaN  1.0   NaN   1.0   NaN  1.0   NaN
dog   NaN   1.0  NaN   1.0   NaN   1.0  NaN   1.0

In [35]:
# 例3 组合是唯一的

classes = [('A', 'cat', 'big'),
           ('A', 'dog', 'small'),
           ('B', 'cat', 'big'),
           ('B', 'cat', 'big')]

m_index1=pd.MultiIndex.from_tuples(classes)
df = pd.DataFrame(np.ones((4,2)),
                 index = m_index1,
                 columns = ['col_1','col_2'])
                
df

col_1  col_2
A cat big      1.0    1.0
  dog small    1.0    1.0
B cat big      1.0    1.0
      big      1.0    1.0

In [36]:
# 例3 组合是唯一的

try:
    df.unstack()
except Exception as e:
    Error_mess = e
    
Error_mess

ValueError('Index contains duplicate entries, cannot reshape')

In [38]:
# 例4 stack


classes = [('A', 'cat', 'big'),
           ('A', 'dog', 'small'),
           ('B', 'cat', 'big'),
           ('B', 'dog', 'small')]

m_index1=pd.MultiIndex.from_tuples(classes)
df = pd.DataFrame(np.ones((4,2)),
                 index = m_index1,
                 columns = ['col_1','col_2']).T
           
df

A          B      
       cat   dog  cat   dog
       big small  big small
col_1  1.0   1.0  1.0   1.0
col_2  1.0   1.0  1.0   1.0

In [39]:
# 例4 stack

df.stack()

A         B     
             cat  dog  cat  dog
col_1 big    1.0  NaN  1.0  NaN
      small  NaN  1.0  NaN  1.0
col_2 big    1.0  NaN  1.0  NaN
      small  NaN  1.0  NaN  1.0

#### 2.2. 聚合与变形的关系

### 三、其他变形函数

#### 3.1. crosstab

* **pd.crosstab**(index,columns,aggfunc)  
    
    计算列之间的数据


* crosstab和pivot_table的**区别和联系**：
    
    * crosstab函数能实现的功能,<font color = red >pivot_table都可以实现</font>，并且pivot_table 性能更快
    
    * crosstab 的对应位置传入的是<font color = red >具体的序列</font>，而 pivot_table 传入的是<font color = red >被调用表对应的名字</font>

In [5]:
df = pd.read_csv('../data/learn_pandas.csv')
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Date,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


In [6]:
# crosstab
pd.crosstab(index = df.School,columns = df.Transfer
           )


Transfer,N,Y
School,,
Fudan University,38,1
Peking University,28,2
Shanghai Jiao Tong University,53,0
Tsinghua University,62,4


In [9]:
# pivot_table
df.pivot_table(columns = 'Transfer',
               index= 'School',
               values= 'Name',
               aggfunc = 'count'
          )

Transfer,N,Y
School,,
Fudan University,38.0,1.0
Peking University,28.0,2.0
Shanghai Jiao Tong University,53.0,NaN
Tsinghua University,62.0,4.0


#### 3.2. explode
#### 3.3. get_dummies

###  四、练习

#### Ex1：美国非法药物数据集


In [31]:
df = pd.read_csv('../data/drugs.csv').sort_values([
    'State','COUNTY','SubstanceName'],ignore_index=True)
df.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2011,KY,ADAIR,Buprenorphine,3
1,2012,KY,ADAIR,Buprenorphine,5
2,2013,KY,ADAIR,Buprenorphine,4
3,2014,KY,ADAIR,Buprenorphine,27
4,2015,KY,ADAIR,Buprenorphine,5


In [35]:
# 参考了答案
res = df.pivot(index=['State','COUNTY','SubstanceName'],
        columns=['YYYY'],
        values = ['DrugReports']).reset_index().rename_axis(columns={'YYYY':''})
res.head()

State COUNTY  SubstanceName DrugReports                                    \
                                     2010 2011  2012  2013  2014 2015  2016   
0    KY  ADAIR  Buprenorphine         NaN  3.0   5.0   4.0  27.0  5.0   7.0   
1    KY  ADAIR        Codeine         NaN  NaN   1.0   NaN   NaN  NaN   NaN   
2    KY  ADAIR       Fentanyl         NaN  NaN   1.0   NaN   NaN  NaN   NaN   
3    KY  ADAIR         Heroin         NaN  NaN   1.0   2.0   NaN  1.0   NaN   
4    KY  ADAIR    Hydrocodone         6.0  9.0  10.0  10.0   9.0  7.0  11.0   

         
   2017  
0  10.0  
1   1.0  
2   NaN  
3   2.0  
4   3.0

#### Ex2：特殊的wide_to_long方法